# Automated ML

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

We start by setting up our experiment in our workspace.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'cc_automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FSTL43BXF to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-135569
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-135569


## Dataset

We're using the [UCI's Machine Learning repository's Cervical cancer (Risk Factors) Data Set](https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29). The dataset was collected at 'Hospital Universitario de Caracas' in Caracas, Venezuela. The dataset comprises demographic information, habits, and historic medical records of 858 patients. Several patients decided not to answer some of the questions because of privacy concerns (missing values).

**Citation: Kelwin Fernandes, Jaime S. Cardoso, and Jessica Fernandes. 'Transfer Learning with Partial Observability Applied to Cervical Cancer Screening.' Iberian Conference on Pattern Recognition and Image Analysis. Springer International Publishing, 2017.**

Our target variable here is **Biopsy**. Biopsy is a sample of tissue taken from the body in order to examine it more closely. A doctor should recommend a biopsy when an initial test suggests an area of tissue in the body isn't normal. Doctors may call an area of abnormal tissue a lesion, a tumor, or a mass.
Here this categorical variable contains the *Biopsy Test Result*.

More information about how the data is cleaned and used maybe found in the `data-cleaning.ipynb` and `train.py` files.

In [3]:
ds = TabularDatasetFactory.from_delimited_files(path="https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv")

Here, we clean and observe our data.

In [4]:
x, y = clean_data(ds)

df = pd.concat([x, pd.DataFrame(y)], axis = 1)
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:molluscum contagiosum,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Biopsy
0,18.0,4.0,15.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,17.089779,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.000000,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,46.0,3.0,21.000000,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we split our data in order to fe

In [6]:
outname2='training_dataset.csv'
outdir2='training/'
if not os.path.exists(outdir2):
    os.mkdir(outdir2)

df_train, df_test = train_test_split(df, test_size=0.25)

fullpath2=os.path.join(outdir2,outname2)
df_test.to_csv(fullpath2)

outname='validation_dataset.csv'
outdir='validation/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullpath=os.path.join(outdir,outname)
df_test.to_csv(fullpath)

Now we store our dataset in our default datastore in order to access it.

In [7]:
datastore = ws.get_default_datastore()

In [8]:
datastore.upload(src_dir = "training/", target_path = "data/")
datastore.upload(src_dir = "validation/", target_path = "data/")

Uploading an estimated of 1 files
Uploading training/training_dataset.csv
Uploaded training/training_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading validation/validation_dataset.csv
Uploaded validation/validation_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_b46be8560a2943de951148d4e0ac9608

In [9]:
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_dataset.csv"))])
validation_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/validation_dataset.csv"))])

## AutoML Configuration

We start by setting up our compute cluster, where we will run our automl run.

In [10]:
cpu_cluster_name = "cpucluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


We've used the following configuration for our run:

|Setting |Reasons|
|-|-|
|**experiment_timeout_minutes**| Maximum amount of time in minutes that all iterations combined can take before the experiment terminates. I've taken this to be 30 mins due to the presence of 730 rows. |
|**max_concurrent_iterations**|These are the iterations occuring simultaneously and has to be equal to the number of nodes in the cluster(5-1))|
|**n_cross_validations**|Using 5 cross validations to avoi8d overfitting) |
|**primary_metric**|Since the data isn't quite balanced, Weighted Average Precision Score |
|**task**|Classification |
|**compute_target**|This is the compute cluster we will be using |
|**training_data**|This is the training dataset stored in the default datastore  |
|**label_column_name**|This is the target variable|


In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" :30,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'average_precision_score_weighted',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    n_cross_validations=5,
    task="classification",
    primary_metric="average_precision_score_weighted",
    compute_target=cpu_cluster,
    training_data=training_data,
    label_column_name="Biopsy",
    max_cores_per_iteration=-1
    )

Submitting the run

In [12]:
remote_run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster
Parent Run ID: AutoML_1f3a196d-618a-4b9d-ae30-76a5b8ebf0ef

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------

## Run Details
The `Rundetails` widget, as the name suggests gives us greater insight about how the Run is proceeding, enabling us to monitor and understand the situation, thereby dealing with it accordingly.

In [14]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

The best performing model is the `VotingEnsemble` with a score of 0.9150. It maybe observed to be derived from the following:

* "ensembled_iterations": "[11, 13, 14, 15, 8, 4, 5, 6, 9, 0]",
* "ensembled_algorithms": "['ExtremeRandomTrees', 'ExtremeRandomTrees', 'RandomForest', 'XGBoostClassifier', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'RandomForest', 'ExtremeRandomTrees', 'RandomForest', 'LightGBM']",
* "ensemble_weights": "[0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667]",
* "best_individual_pipeline_score": "0.9112431387013187",

In [15]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Cervical_Cancer_prediction_Biopsy_auto_ml', model_path='./')

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: cc_automl,
Id: AutoML_1f3a196d-618a-4b9d-ae30-76a5b8ebf0ef_19,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                silent=True,
                                                                                                subsample=1.0,
                                                 

## Model Deployment

Being the better performing model, I shall hereby deploy the `VotingEnsemble` model.

In [16]:
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_cc.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
from azureml.core.model import Model
model=best_run.register_model(
            model_name = 'automl-bestmodel-cc', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Cervical Cancer Prediction'
)

In [19]:
from azureml.core import Environment

# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [20]:
from azureml.core.model import InferenceConfig

# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_cc.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_cc.py',
                                    environment=myenv)

In [21]:
from azureml.core.webservice import AciWebservice

# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [51]:
service = Model.deploy(workspace=ws, 
                       name='automl-bestmodel-cc', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [52]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-135569', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-135569'), name=automl-bestmodel-cc, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://9f77366b-ca32-49d8-b59a-4a4db6051336.southcentralus.azurecontainer.io/score, properties=None, created_by={})

In [53]:

# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://9f77366b-ca32-49d8-b59a-4a4db6051336.southcentralus.azurecontainer.io/score
Swagger URI:
http://9f77366b-ca32-49d8-b59a-4a4db6051336.southcentralus.azurecontainer.io/swagger.json


In [59]:
import json

# select 3  samples from the dataframe
x_df=df.sample(3)
y_df = x_df.pop('Biopsy')

x_df['Column1'] = 0.0

# convert the records to a json data file
recored=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recored})
print(scoring_json)

{"data": [{"Age": 20.0, "Number of sexual partners": 2.0, "First sexual intercourse": 16.0, "Num of pregnancies": 2.316400580551524, "Smokes": 0.0, "Smokes (years)": 0.0, "Smokes (packs/year)": 0.0, "Hormonal Contraceptives": 1.0, "Hormonal Contraceptives (years)": 0.75, "IUD": 0.0, "IUD (years)": 0.0, "STDs": 0.0, "STDs (number)": 0.0, "STDs:condylomatosis": 0.0, "STDs:vaginal condylomatosis": 0.0, "STDs:vulvo-perineal condylomatosis": 0.0, "STDs:syphilis": 0.0, "STDs:pelvic inflammatory disease": 0.0, "STDs:genital herpes": 0.0, "STDs:molluscum contagiosum": 0.0, "STDs:HIV": 0.0, "STDs:Hepatitis B": 0.0, "STDs:HPV": 0.0, "STDs: Number of diagnosis": 0.0, "Dx:Cancer": 0.0, "Dx:CIN": 0.0, "Dx:HPV": 0.0, "Dx": 0.0, "Column1": 0.0}, {"Age": 45.0, "Number of sexual partners": 5.0, "First sexual intercourse": 15.0, "Num of pregnancies": 7.0, "Smokes": 0.0, "Smokes (years)": 0.0, "Smokes (packs/year)": 0.0, "Hormonal Contraceptives": 1.0, "Hormonal Contraceptives (years)": 0.66, "IUD": 0.0,

In [60]:
output = service.run(scoring_json)
output

'{"result": [0.0, 0.0, 0.0]}'

In [61]:
y_df

365   0.00
588   0.00
16    0.00
Name: Biopsy, dtype: float64

In [62]:
print(service.get_logs())

2021-01-21T10:45:36,305782600+00:00 - rsyslog/run 
2021-01-21T10:45:36,303537300+00:00 - iot-server/run 
2021-01-21T10:45:36,350405100+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-21T10:45:36,378756300+00:00 - nginx/run 
rsyslogd

In [63]:
service.update(enable_app_insights=True)

In [66]:
service.delete

<bound method Webservice.delete of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-135569', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-135569'), name=automl-bestmodel-cc, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://9f77366b-ca32-49d8-b59a-4a4db6051336.southcentralus.azurecontainer.io/score, properties=None, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})>